In [1]:
'''
根据产成品出入库流水账统计废纸房产品入库数量
'''
import os
import datetime
import easygui
import numpy as np
import pandas as pd
import openpyxl
from openpyxl.styles import Font, Border, Side, Fill, Alignment
import formatPainter
import re

In [2]:
dic = {'010201': '软抄',
 '010213': '定制产品',
 '010211': '定制产品',
 '010212': '莱特牛皮缝线',
 '010301': '锐意缝线本',
 '010311': '文稿纸',
 '010303': '锐意软抄',
 '010308': '锐意专利作业本',
 '010304': '锐意专利牛皮本',
 '010309': '锐意处理产品',
 '010310': '锐意无线胶装',
 '010501': '新锐缝线',
 '010503': '新锐软抄',
 '010504': '新锐防近视',
 '010599': '新锐定制（原木）',
 '010598': '新锐处理产品',
 '010606': '电商缝线',
 '010607': '电商抄本',
 '010608': '定制台板缝线',
 '0107': '外贸',
 '010602': '备课本',
 '010305': '锐意美术本',
 '010601': '电商美术本'}

col_lst = ['单据日期','业务类型','单据编号','单据类型','仓库','存货分类编码','存货分类','存货编码','存货','规格型号','入库数量',
 '入库数量2',
 '入库计量单位组合',
 '入库金额',
 '出库数量',
 '出库数量2']

# fname = r"F:\a00nutstore\008\zw08\废纸房\出入库流水9.26-10.25导出数据.xlsx"
msg = '请点选出入库流水'
fname = easygui.fileopenbox(msg)
path,_ = os.path.split(fname)
os.chdir(path)
df = pd.read_excel(fname, skiprows = 7,dtype= {'Unnamed: 6':str})
df = df.iloc[:, 1:]
df

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,入库数量,入库数量2,入库计量单位组合,入库金额,出库数量,出库数量2
0,2024-09-29,其他退库,IC-2024-09-0124,其他入库单,发货仓库,010501,新锐缝线本,RFDS-1660,16K60型缝线卡面读书笔记,D1新锐缝线本,-364.0,-1.82,-1件-164本,NaN,NaN,NaN
1,2024-09-29,其他退库,IC-2024-09-0125,其他入库单,发货仓库,01020825,山东版小学生本,SD202Z,16K英语本,B8小学生作业本,-2390.0,-11.95,-11件-190本,NaN,NaN,NaN
2,2024-09-29,其他退库,IC-2024-09-0125,其他入库单,发货仓库,01020825,山东版小学生本,SD203Z,16K笔记本,B8小学生作业本,-800.0,-4.00,-4件,NaN,NaN,NaN
3,2024-09-29,其他退库,IC-2024-09-0125,其他入库单,发货仓库,01020825,山东版小学生本,SD204Z,16K作文本,B8小学生作业本,-990.0,-4.95,-4件-190本,NaN,NaN,NaN
4,2024-09-29,其他退库,IC-2024-09-0125,其他入库单,发货仓库,01020825,山东版小学生本,SD206Z,16K田格本,B8小学生作业本,-400.0,-2.00,-2件,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4165,制表人:,黄锦,NaN,打印日期:,2024-11-22,NaN,其他信息:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.columns = col_lst
df = df[~df['存货编码'].isnull()]
df1 = df[['存货分类编码','存货编码','存货','规格型号','入库数量2']]
df1

,存货分类编码,存货编码,存货,规格型号,入库数量2
0,010501,RFDS-1660,16K60型缝线卡面读书笔记,D1新锐缝线本,-1.82
1,01020825,SD202Z,16K英语本,B8小学生作业本,-11.95
2,01020825,SD203Z,16K笔记本,B8小学生作业本,-4.00
3,01020825,SD204Z,16K作文本,B8小学生作业本,-4.95
4,01020825,SD206Z,16K田格本,B8小学生作业本,-2.00
...,...,...,...,...,...
4158,01060504,EJL16150-B01,16K150型卡面胶套本,B19电商胶套本,NaN
4159,01060503,EJL32150-B01,32K150型卡面胶套本,B19电商胶套本,NaN
4160,01010201,1105,16K现金日记账,A2账芯,NaN
4161,01010201,1106,16K三栏式明细分类账,A2账芯,NaN


In [5]:
df2 = df1.assign(feipin = (
 df1['存货分类编码']
    .str[:6]
    # .map(lambda x:x.zfill(6))
    .map(dic)
))
df2



,存货分类编码,存货编码,存货,规格型号,入库数量2,feipin
0,010501,RFDS-1660,16K60型缝线卡面读书笔记,D1新锐缝线本,-1.82,新锐缝线
1,01020825,SD202Z,16K英语本,B8小学生作业本,-11.95,NaN
2,01020825,SD203Z,16K笔记本,B8小学生作业本,-4.00,NaN
3,01020825,SD204Z,16K作文本,B8小学生作业本,-4.95,NaN
4,01020825,SD206Z,16K田格本,B8小学生作业本,-2.00,NaN
...,...,...,...,...,...,...
4158,01060504,EJL16150-B01,16K150型卡面胶套本,B19电商胶套本,NaN,NaN
4159,01060503,EJL32150-B01,32K150型卡面胶套本,B19电商胶套本,NaN,NaN
4160,01010201,1105,16K现金日记账,A2账芯,NaN,NaN
4161,01010201,1106,16K三栏式明细分类账,A2账芯,NaN,NaN


In [6]:
df3 = df2[~df2.feipin.isnull()]
df3.sum()
df3

,存货分类编码,存货编码,存货,规格型号,入库数量2,feipin
0,010501,RFDS-1660,16K60型缝线卡面读书笔记,D1新锐缝线本,-1.82,新锐缝线
8,010501,RFDS-1660,16K60型缝线卡面读书笔记,D1新锐缝线本,NaN,新锐缝线
18,0106060202,EFPY1660N,16K60型牛卡缝线拼音本,B20电商缝线本,9.00,电商缝线
22,0106060202,EFY1660NB,16K60型牛卡缝线英语本,B20电商缝线本,77.00,电商缝线
29,0106060202,EFTZ1660N（10本）,16K60型牛卡缝线田字本（10本装）,B20电商缝线本,100.00,电商缝线
...,...,...,...,...,...,...
4150,01060801,EFL1660K,16K60型卡面五彩马卡龙缝线练习本,B22电商定制,-1.00,定制台板缝线
4151,01060801,EFSX1660K,16K60型卡面五彩马卡龙缝线数学本,B22电商定制,-1.00,定制台板缝线
4152,01060801,EFY1660K,16K60型卡面五彩马卡龙缝线英语本,B22电商定制,-1.00,定制台板缝线
4153,01060801,EFZW1660K,16K60型卡面五彩马卡龙缝线作文本,B22电商定制,-1.00,定制台板缝线


In [9]:
result = pd.pivot_table(df3,index = 'feipin',values = '入库数量2',aggfunc = sum)
result.loc['合计','入库数量2'] = result['入库数量2'].sum()
result

C:\Users\redda\AppData\Local\Temp\ipykernel_14148\713679194.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  result = pd.pivot_table(df3,index = 'feipin',values = '入库数量2',aggfunc = sum)


,入库数量2
feipin,
备课本,0.380
外贸,4937.000
定制产品,566.222
定制台板缝线,242.497
文稿纸,-5.000
新锐定制（原木）,0.000
新锐缝线,317.927
新锐软抄,192.995
新锐防近视,205.000


In [10]:
dic1 = {v:k for k,v in dic.items()}
result = result.assign(biama = (
    result.index
    .map(dic1)
))
result = result.sort_values('biama')
result = result[['入库数量2']]
result.index.name = '产品大类'
result.columns =  ['单位（件）']
result


,单位（件）
产品大类,
软抄,5889.908
定制产品,566.222
莱特牛皮缝线,1737.950
锐意缝线本,35.650
锐意软抄,-15.021
锐意专利牛皮本,453.000
锐意美术本,-11.727
锐意处理产品,6.996
锐意无线胶装,-6.280


In [12]:
result.to_excel('废品入库数量.xlsx',index = True)

In [13]:
'''
根据产成品出入库流水账统计废纸房产品入库数量
'''
import os
import datetime
import easygui
import numpy as np
import pandas as pd
import openpyxl
from openpyxl.styles import Font, Border, Side, Fill, Alignment
import formatPainter
import re

dic = {'010201': '软抄',
 '010213': '定制产品',
 '010211': '定制产品',
 '010212': '莱特牛皮缝线',
 '010301': '锐意缝线本',
 '010311': '文稿纸',
 '010303': '锐意软抄',
 '010308': '锐意专利作业本',
 '010304': '锐意专利牛皮本',
 '010309': '锐意处理产品',
 '010310': '锐意无线胶装',
 '010501': '新锐缝线',
 '010503': '新锐软抄',
 '010504': '新锐防近视',
 '010599': '新锐定制（原木）',
 '010598': '新锐处理产品',
 '010606': '电商缝线',
 '010607': '电商抄本',
 '010608': '定制台板缝线',
 '0107': '外贸',
 '010602': '备课本',
 '010305': '锐意美术本',
 '010601': '电商美术本'}

col_lst = ['单据日期','业务类型','单据编号','单据类型','仓库','存货分类编码','存货分类','存货编码','存货','规格型号','入库数量',
 '入库数量2',
 '入库计量单位组合',
 '入库金额',
 '出库数量',
 '出库数量2']

# fname = r"F:\a00nutstore\008\zw08\废纸房\出入库流水9.26-10.25导出数据.xlsx"
msg = '请点选出入库流水'
fname = easygui.fileopenbox(msg)
path,_ = os.path.split(fname)
os.chdir(path)
df = pd.read_excel(fname, skiprows = 7,dtype= {'Unnamed: 6':str})
df = df.iloc[:, 1:]
df








,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,入库数量,入库数量2,入库计量单位组合,入库金额,出库数量,出库数量2
0,2024-09-29,其他退库,IC-2024-09-0124,其他入库单,发货仓库,010501,新锐缝线本,RFDS-1660,16K60型缝线卡面读书笔记,D1新锐缝线本,-364.0,-1.82,-1件-164本,NaN,NaN,NaN
1,2024-09-29,其他退库,IC-2024-09-0125,其他入库单,发货仓库,01020825,山东版小学生本,SD202Z,16K英语本,B8小学生作业本,-2390.0,-11.95,-11件-190本,NaN,NaN,NaN
2,2024-09-29,其他退库,IC-2024-09-0125,其他入库单,发货仓库,01020825,山东版小学生本,SD203Z,16K笔记本,B8小学生作业本,-800.0,-4.00,-4件,NaN,NaN,NaN
3,2024-09-29,其他退库,IC-2024-09-0125,其他入库单,发货仓库,01020825,山东版小学生本,SD204Z,16K作文本,B8小学生作业本,-990.0,-4.95,-4件-190本,NaN,NaN,NaN
4,2024-09-29,其他退库,IC-2024-09-0125,其他入库单,发货仓库,01020825,山东版小学生本,SD206Z,16K田格本,B8小学生作业本,-400.0,-2.00,-2件,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4165,制表人:,黄锦,NaN,打印日期:,2024-11-22,NaN,其他信息:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
len(df.columns)

16

In [16]:
len(col_lst)

16